### Imports

In [ ]:
# !pip install langchain_google_genai
# !pip install langchain_community
# !pip install langchain_huggingface
# !pip install gradio
# !pip install rapidfuzz
# !pip install pypdf
# !pip install faiss-cpu
# !pip install spacy
# !pip install fuzzywuzzy
# !python -m spacy download en_core_web_sm
# !pip install python-Levenshtein

  Using cached langchain_google_genai-2.1.2-py3-none-any.whl.metadata (4.7 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached google_ai_generativelanguage-0.6.17-py3-none-any.whl.metadata (9.8 kB)
  Using cached langchain_core-0.3.51-py3-none-any.whl.metadata (5.9 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached langsmith-0.3.30-py3-none-any.whl.metadata (15 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.1-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.30.2 which is incompatible.


  Using cached langchain_community-0.3.21-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain-0.3.23-py3-none-any.whl.metadata (7.8 kB)
  Using cached sqlalchemy-2.0.40-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached aiohttp-3.11.16-cp312-cp312-win_amd64.whl.metadata (8.0 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.8.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.4.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached propcache-0.3.1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached yarl-1.19.0-cp312-cp312-win_amd64.whl.metadata (74 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.meta

In [ ]:
# !pip install django
# !pip install django-cors-headers
# !pip install djangorestframework
# !pip install drf_spectacular
# !pip install psycopg2

  Using cached psycopg2-2.9.10-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
Using cached psycopg2-2.9.10-cp312-cp312-win_amd64.whl (1.2 MB)


In [ ]:
# !python.exe -m pip install --upgrade pip

In [1]:
import json
import subprocess
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
import re, ast
import gradio as gr
from sentence_transformers import SentenceTransformer, util
import spacy
from rapidfuzz import fuzz
pdf_file = r"C:\Users\a1mme\OneDrive\Desktop\MO\test_grad\chatbot\res\Cli script guidebook.pdf"


d:\MO\Depi\envai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Model

In [11]:
def get_llm(model_name="gemini-1.5-pro"):
    return ChatGoogleGenerativeAI(
        model=model_name,
        temperature=0.1,
        max_output_tokens=500,
        google_api_key="AIz..."
    )

Load & Prepare Data

In [13]:
loader = PyPDFLoader(pdf_file)
pdf_docs = loader.load_and_split()

# Load mapping
with open("res/mapping.json", "r") as f:
    raw_mapping = json.load(f)

with open("res/data_mapping.json", "r") as f:
    data_mapping = json.load(f)

# Format mapping as Document
mapping_text = "\n".join(f"{k} -> {v}" for k, v in raw_mapping.items())
mapping_doc = Document(page_content=mapping_text)

all_docs = pdf_docs + [mapping_doc]

### Create Vector Space

In [14]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embeddings)
retriever = vectorstore.as_retriever()

### RAG Chain

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def combine_inputs(input_dict):
    question = input_dict["question"]
    retrieved_docs = retriever.get_relevant_documents(question)
    return {
        "context": format_docs(retrieved_docs),
        "question": question
    }

template = """You are an expert assistant trained to extract exact command-line instructions from a user guide.

You are also provided with a node-to-subcommand mapping reference (e.g., "logistic_regression" → "model", "standard_scaler" → "preprocessor"). Use this mapping only to determine the correct value for <node_name> in the CLI command syntax.

Reference Information:
{context}

User Query:
{question}

Instructions:
- Carefully read the Reference Information, including any node-to-subcommand mappings and CLI usage documentation.
- Return the CLI command(s) that are relevant to the user's query with the node name in the node-to subcommand mapping.
- Use the mapping to replace <node_name> with its corresponding type (e.g., model, preprocessor, etc.)
- IMPORTANT: Never modify or replace <args>. Keep <args> exactly as it is shown.
- Format your response as a raw Python list: ['command1', 'command2', ...]
- Do NOT include any explanations, comments, or formatting outside the list.
- If no command matches the query, return an empty list: []
- The output MUST be ordered in the same order as the input.

Example Query:
I want to make a logistic regression model.

Expected Response:
[
    ('make model <args>', 'logistic_regression')
]
"""

prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    RunnablePassthrough()
    | combine_inputs
    | prompt
    | get_llm(model_name="gemini-2.0-flash")
    | StrOutputParser()
)

### Keyword Extraction (Hybrid)

In [16]:
name_extractor = SentenceTransformer('all-MiniLM-L6-v2')
nlp = spacy.load("en_core_web_sm")
REFERENCE_KEYWORDS = list(raw_mapping.keys())
keyword_embeddings = name_extractor.encode(REFERENCE_KEYWORDS, convert_to_tensor=True)

def extract_keywords_hybrid(user_input, transformer_thresh=0.7, fuzzy_thresh=80):
    doc = nlp(user_input.lower())
    phrases = [chunk.text for chunk in doc.noun_chunks]
    matched_keywords = []

    for phrase in phrases:
        phrase_embedding = name_extractor.encode(phrase, convert_to_tensor=True)
        cosine_scores = util.cos_sim(phrase_embedding, keyword_embeddings)[0]

        for idx, score in enumerate(cosine_scores):
            if score >= transformer_thresh:
                if REFERENCE_KEYWORDS[idx] not in matched_keywords:
                    matched_keywords.append(REFERENCE_KEYWORDS[idx])

        for keyword in REFERENCE_KEYWORDS:
            if fuzz.ratio(phrase, keyword.replace("_", " ")) >= fuzzy_thresh:
                if keyword not in matched_keywords:
                    matched_keywords.append(keyword)

    return matched_keywords

### Post-Processing

In [17]:
def parse_command_list(output: str):
    pattern = r"\[(.*?)\]"
    matches = re.findall(pattern, output, re.DOTALL)

    if matches:
        output = f"[{matches[0]}]"

    try:
        command_list = ast.literal_eval(output.strip())
        return command_list if isinstance(command_list, list) else [command_list]
    except Exception as e:
        return [f"Failed to parse: {e}"]

def args_extractor(names, mapping, data_mapping):
    result = []
    for name in names:
        if name in mapping and name in data_mapping:
            node_type = mapping[name]
            args_str = json.dumps(data_mapping[name], separators=(',', ':'))
            result.append((node_type, args_str))
    return result

def replace_args(commands, replacements):
    for i in range(len(commands)):
        commands[i] = commands[i].replace("<node_name>", replacements[i][0]).replace("<args>", replacements[i][1])
    return commands

### Run Everything

In [20]:
question = "please add a standard scaler preprocessor, and a linear regressin and logistic regression."

# RAG output
rag_output = rag_chain.invoke({"question": question})

# Extract command names
names = extract_keywords_hybrid(question)

# Extract args and types
output_list = args_extractor(names, raw_mapping, data_mapping)

# Parse RAG and replace placeholders
parsed_output = parse_command_list(rag_output)
final_output = replace_args(parsed_output, output_list)
final_output

['make preprocessor {"preprocessor_name":"standard_scaler","preprocessor_type":"scaler","params":{"with_mean":true,"with_std":true}}',
 'make model {"model_name":"linear_regression","task":"regression","model_type":"linear_models","params":{}}',
 'make model {"model_name":"logistic_regression","task":"classification","model_type":"linear_models","params":{"penalty":"l2","C":1.0}}']